# 200812 Fastq kmer search gold standard genomes

In [1]:
using BioSequences
using FASTX
using CodecZlib
using ProgressMeter
using DataFrames
using HDF5

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-12810070761407346754\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16285707294156897813\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-12810070761407346754\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
using Midas.Kmers
using Midas.Search

## Configuration and directories

In [4]:
infiles = Dict(
    :assembly_dir => "/Users/student/projects/midas/data/gold_standard_seqs_200726/assembled",
    :reads_dir => "/Users/student/projects/midas/data/gold_standard_seqs_200726/reads",
)

Dict{Symbol,String} with 2 entries:
  :assembly_dir => "/Users/student/projects/midas/data/gold_standard_seqs_20072…
  :reads_dir    => "/Users/student/projects/midas/data/gold_standard_seqs_20072…

In [18]:
tmpdir = "tmp/"

isdir(tmpdir) || mkdir(tmpdir)

"tmp/"

## Find sequence files

In [6]:
seq_files = let
    readfiles = filter(f -> endswith(f, ".fastq.gz"), readdir(infiles[:reads_dir], join=true))

    pairs = map(readfiles) do readsfile
        key = split(basename(readsfile), '.')[1]   
        assemblyfile = joinpath(infiles[:assembly_dir], key * ".fasta")
        return key => (raw=readsfile, assembled=assemblyfile)
    end

    Dict(pairs)
end

for (raw, assembled) in values(seq_files)
    @assert isfile(raw) && isfile(assembled)
end

length(seq_files)

80

## Defs

In [7]:
phred2p(phred) = 10 ^ (phred * -0.1)
p2phred(p) = -10 * log10(p)
phredprod(phred) = p2phred(1 - prod(1 .- phred2p.(phred)))
phredsum(phred) = p2phred(sum(phred2p.(phred)))

phredsum (generic function with 1 method)

## Kmer searches

### Parameters

In [9]:
kspec = KmerSpec{11}(kmer"ATGAC")
query = KmerQuery(kspec)

NKMERS = Int(nkmers(kspec.k))
K = Midas.Search.matchtype(query)
U = BioSequences.encoded_data_type(K)

UInt32

In [10]:
score_thresholds = [10, 15, 20, 25];

### Setup

In [11]:
datafiles = Dict(key => joinpath(tmpdir, key * ".h5") for key in keys(seq_files));

In [12]:
# Preallocate arrays
raw_counts = zeros(Int, length(score_thresholds), nkmers(kspec.k));
assembly_counts = zeros(Int, nkmers(kspec.k));

### Run

In [19]:
for (i, key) in enumerate(sort(collect(keys(seq_files))))
    datafile = datafiles[key]
    isfile(datafile) && continue
    
    pbar = ProgressUnknown(desc="$i/$(length(seq_files)) ")
    
    # Keep in tmp directory until were done writing to it
    tmpfile = tempname()
    h5 = h5open(tmpfile, "cw")
    
    # Assembly
    let
        assembly_counts .= 0
        contig_lengths = Int[]
        
        g = g_create(h5, "assembly")
        
        fasta = open(seq_files[key].assembled)
        reader = FASTA.Reader(fasta)

        for record in reader
            seq = sequence(record)
            push!(contig_lengths, length(seq))

            for m in findkmers(query, seq)
                ikmer = BioSequences.encoded_data(m.kmer) + 1
                assembly_counts[ikmer] += 1
            end
        end
        
        ikmers = findall(>(0), assembly_counts)
        
        # Write results
        g["kmers"] = U.(ikmers .- 1)
        g["counts"] = assembly_counts[ikmers]
        g["contig_lengths"] = contig_lengths
    end
    
    # Reads
    let
        raw_counts .= 0
        nreads = 0
        total_len = 0
        
        g = g_create(h5, "raw")
        
        fastq = GzipDecompressorStream(open(seq_files[key].raw))
        reader = FASTQ.Reader(fastq)
        
        for record in reader
            seq = sequence(record)
            phred = quality(record)
            
            nreads += 1
            total_len += length(seq)

            for m in findkmers(query, seq)
                score = phredprod(phred[m.fullinds])
                ithresh = searchsortedfirst(score_thresholds, score) - 1
                ithresh < 1 && continue

                ikmer = BioSequences.encoded_data(m.kmer) + 1
                raw_counts[1:ithresh, ikmer] .+= 1
            end

            next!(pbar)
        end
        
        ikmers = [i for (i, c) in enumerate(eachcol(raw_counts)) if any(>(0), c)]
        
        # Write results
        g["kmers"] = U.(ikmers .- 1)
        g["counts"] = raw_counts[:, ikmers]
        g["score_thresholds"] = score_thresholds
        attrs(g)["nreads"] = nreads
        attrs(g)["mean_read_length"] = total_len / nreads
    end
    
    # Finish
    close(h5)
    mv(tmpfile, datafile)
    finish!(pbar)
end

1/80  1814188 	 Time: 0:00:39
2/80  1721470 	 Time: 0:00:35
3/80  1054659 	 Time: 0:00:21
4/80  2009335 	 Time: 0:00:39
5/80  1340197 	 Time: 0:00:26
6/80  4778804 	 Time: 0:01:32
7/80  3432370 	 Time: 0:01:03
8/80  3864669 	 Time: 0:01:08
9/80  4657878 	 Time: 0:01:23
10/80  2663277 	 Time: 0:00:51
11/80  4224840 	 Time: 0:01:27
12/80  3057451 	 Time: 0:00:58
13/80  2053910 	 Time: 0:00:37
14/80  2252990 	 Time: 0:00:38
15/80  2226237 	 Time: 0:00:39
16/80  3379602 	 Time: 0:01:02
17/80  1270772 	 Time: 0:00:25
18/80  1625017 	 Time: 0:00:31
19/80  1440500 	 Time: 0:00:26
20/80  1978216 	 Time: 0:00:37
21/80  1332848 	 Time: 0:00:25
22/80  890126 	 Time: 0:00:16
23/80  569846 	 Time: 0:00:11
24/80  2275970 	 Time: 0:00:43
25/80  956142 	 Time: 0:00:19
26/80  1633599 	 Time: 0:00:30
27/80  2377593 	 Time: 0:00:43
28/80  1651246 	 Time: 0:00:32
29/80  2215327 	 Time: 0:00:42
30/80  1934372 	 Time: 0:00:44
31/80  2841210 	 Time: 0:00:58
32/80  3030748 	 Time: 0:00:55
33/80  1085060 	 Tim